# Shor's factoring algorithm
This notebook exemplifies using [Quaspy](https://github.com/ekera/quaspy) to simulate Shor's general factoring algorithm [[Shor94]](https://doi.org/10.1109/SFCS.1994.365700), modified as in [[E22p]](https://doi.org/10.48550/arXiv.2201.07791), and with the classical post-processing in [[E21b]](https://doi.org/10.1007/s11128-021-03069-1) and [[E22p]](https://doi.org/10.48550/arXiv.2201.07791).

To start off, let us pick $n$ prime factors $p_i$ uniformly at random from the set of all $l$-bit prime factors, and $n$ exponents $e_i$ uniformly at random from $[1, e_{\max}] \cap \mathbb Z$, for $i \in [1, n] \cap \mathbb Z$, and take the product to form

$$N = \prod_{i=1}^n p_i^{e_i}.$$

To this end, we use the [<code>sample_l_bit_prime(l)</code>](../docs/math/primes/sample_l_bit_prime.md) and [<code>sample_integer(B)</code>](../docs/math/random/sample_integer.md) convenience functions provided by [Quaspy](https://github.com/ekera/quaspy).

In [38]:
from quaspy.math.primes import sample_l_bit_prime;
from quaspy.math.random import sample_integer;

from math import prod;

n = 10;
l = 1024;
e_max = 1;

factors = [];
primes = set();

while len(factors) < n:
  pi = sample_l_bit_prime(l);
  ei = sample_integer(e_max) + 1;

  if pi not in primes:
    primes.add(pi);
    factors.append([pi, ei]);

    print("Sampled p" + str(len(factors)) + " =", pi);

N = prod([pi ** ei for [pi, ei] in factors]);

print("\nComputed N =", N);

Sampled p1 = 95530818810920301323859954721354145641670346076988832890618641281060557058014973554951390168583823688471547392871463828302760319229799523856036766929176887827734618137680833804661535495451153382843942006931810404176944464241129459712370074271324390660380728174818825782833957033171113651225663076780920467849
Sampled p2 = 163395858263974426683516634018315180809697286822239606433063798628835374306777501512860012488351215772476465160633382371142293633205027737553467116150530910178673132686721107413079656998117181340832708632893358822750636465213826756902982164240404092583300367534968075742085015779628808257019481926851139899007
Sampled p3 = 103055157400235427840669600143619699250124560180157035605749748323954549244160687872155603633709796810317834134487432900769743360994069544658982655332933534020105855832950236983389881786447418127984225689523526990670029065531496425005456297389385589013862849512821828383796361951429488361272472907420974641399
Sampled p4 = 9785754340792710822

## 1. Simulating order finding in $\mathbb Z_N^*$ exactly to sample $r$
[Quaspy](https://github.com/ekera/quaspy) provides a function [<code>sample_r_given_N(N, factors)</code>](../docs/factoring/sampling/sample_r_given_N.md) for exactly sampling an element $g$ uniformly at random from $\mathbb Z_N^*$ and returning its order without explicitly computing and returning $g$. This function requires that the factorization of $N$ is known, as is the case here given that we selected the factors of $N$.

Below, we use this function to simulate the initial classical pre-processing step of Shor's algorithm where an element $g$ is selected uniformly at random from $\mathbb Z_N^*$.

In [39]:
from quaspy.factoring.sampling import sample_r_given_N;

r = sample_r_given_N(N, factors);

print("Sampled r =", r);

Sampled r = 7162335469043901171088968610631478054224785319439699251571913511061111436672656836195011745374824587123097814864076632960955792780681982813436199186285488038089829855116068511967455173904603105374173604126709678172802622752701446796051230822938357212054272477366333903387699848787606205055345345025486025696069017231979481559398384756819372547729598398370364040231920666193281556675859918909509216553396152830716168555510366937419798231713045730829020121631617179982276692723886542063219246811885346164005588592282687008021546841541557285612217134023148926955623459535542128155825450063928859491280805191264073046919123894712134518181989367704394527122373943349734821955750175086576271412331695248948162577007928391314729441490174988230261634628891008313523849700537162155441829667469570242536863365693638689619466939323524496938451508929739634849792808818482938490492431676548029267590258050199844150858291987864939950650713027945602690576408712086045634791610795155880445258220142161025

### 1.1. Solving $r$ for the complete factorization of $N$
We may then use the [<code>solve_r_for_factors(r, N, ..)</code>](../docs/factoring/general/postprocessing/ekera/solve_r_for_factors.md) function provided by [Quaspy](https://github.com/ekera/quaspy) to solve $r$ for the factors of $N$ by using the post-processing from [[E21p]](https://doi.org/10.1007/s11128-021-03069-1):

In [40]:
from quaspy.factoring.general.postprocessing.ekera import solve_r_for_factors;

result = solve_r_for_factors(r, N);

print("Solving for the prime factors yielded:", result);

if result == primes:
  print("[ OK ] All prime factors of N were successfully recovered.");
else:
  print("[FAIL] The integer N was not completely factored.");

Solving for the prime factors yielded: {95530818810920301323859954721354145641670346076988832890618641281060557058014973554951390168583823688471547392871463828302760319229799523856036766929176887827734618137680833804661535495451153382843942006931810404176944464241129459712370074271324390660380728174818825782833957033171113651225663076780920467849, 97857543407927108223976823666216924412885200224964459135278027351739401694392321676669136985358183756653565248925947644724612284283337105672741607915407458372068883562261016667136123365453015285069502858535738893604886644778944039363396403852291342307810410482009805706361658322521650907301518396712507141789, 159597504227174565597698629863466563574714132862427668822594835820203283398809094526979263623109822457236741195185169544775182177049623665555153891296800136064564745412141368327254360766883968539202457353783344315528837573545344644329438989225872602933813276961807069734861025132810023540242969225916636453013, 16835063126273033268753871428

## 2. Simulating the quantum part of Shor's order-finding algorithm to sample $j$
[Quaspy](https://github.com/ekera/quaspy) provides a function [<code>sample_j_given_r(r, m, l, ..)</code>](../docs/orderfinding/general/sampling/sample_j_given_r.md) for simulating Shor's order-finding algorithm exactly (up to arbitrary precision) for a given order $r$. Below, we use the this function to simulate running the algorithm for $r$ with a control register of $m + \ell$ bits.

This yields a frequency $j$ sampled from the probability distribution induced by the algorithm:

In [41]:
from quaspy.orderfinding.general.sampling import sample_j_given_r;

m = N.bit_length() - 1;
l = m; # Ensures that r^2 < 2^(m + l) as r < 2^m;

j = sample_j_given_r(r, m, l);

print("Sampled j =", j);

Sampled j = 9642368019762801950680543376258113904419807124189263751882313718086108623558829836169962248626415597444785663310474261130202961240705520827035483026414866075574938827456965116702502405297077458880656981721223181400473147279915869876637424612224718219867681284747158300198261904631587137079371835164505439743060619858256875981743587356450831654733974961776201063164423276104331342970540855276477704579567331866987599549318936811143115006346463161621554564325613592126841259525534330334726412312785800965207077155169999015644473728267002521722596824982628176163209733447116805162393029078433649666563436158654996985247954566950343239998456488363366550489059327267564881189360898822414828956083641989825070877099176354367335320252067516447989776229557127108394436743758903391846074546623589552388274122255112729167156172342145623027848344752883848364710050668800642945168305698960145593250849322019074805381398527511676289821132664264170189721278017190002935266032167911749236745552243125801

### 2.1. Solving $j$ for the complete factorization of $N$
We now proceed to solve $j$ for the factors of $N$.

To this end, we first use the [SimulatedCyclicGroupElement](../docs/math/groups/SimulatedCyclicGroupElement.md) class provided by [Quaspy](https://github.com/ekera/quaspy) to define a group element $g$ of order $r$:

In [42]:
from quaspy.math.groups import SimulatedCyclicGroupElement;

g = SimulatedCyclicGroupElement(r);

We may then e.g. use the [<code>solve_j_for_factors(j, m, l, g, N, ..)</code>](../docs/factoring/general/postprocessing/ekera/solve_j_for_factors.md) convenience function provided by [Quaspy](https://github.com/ekera/quaspy) to first solve $j$ for a positive integer multiple of $r$ using the lattice-based post-processing from [[E22p]](https://doi.org/10.48550/arXiv.2201.07791), and to then solve $r$ for the prime factors of $N$ using the post-processing in [[E21p]](https://doi.org/10.1007/s11128-021-03069-1):

In [43]:
from quaspy.factoring.general.postprocessing.ekera import solve_j_for_factors;

result = solve_j_for_factors(j, m, l, g, N);

print("Solving for the prime factors yielded:", result);

if result == primes:
  print("[ OK ] All prime factors of N were successfully recovered.");
else:
  print("[FAIL] Failed to completely factor the integer N.");

Solving for the prime factors yielded: {95530818810920301323859954721354145641670346076988832890618641281060557058014973554951390168583823688471547392871463828302760319229799523856036766929176887827734618137680833804661535495451153382843942006931810404176944464241129459712370074271324390660380728174818825782833957033171113651225663076780920467849, 97857543407927108223976823666216924412885200224964459135278027351739401694392321676669136985358183756653565248925947644724612284283337105672741607915407458372068883562261016667136123365453015285069502858535738893604886644778944039363396403852291342307810410482009805706361658322521650907301518396712507141789, 159597504227174565597698629863466563574714132862427668822594835820203283398809094526979263623109822457236741195185169544775182177049623665555153891296800136064564745412141368327254360766883968539202457353783344315528837573545344644329438989225872602933813276961807069734861025132810023540242969225916636453013, 16835063126273033268753871428

We may also solve in two steps, by first calling [<code>solve_j_for_r(j, m, l, g, ..)</code>](../docs/orderfinding/general/postprocessing/ekera/solve_j_for_r.md) to first solve $j$ for a positive integer multiple of $r$ using the lattice-based post-processing from [[E22p]](https://doi.org/10.48550/arXiv.2201.07791), and by then calling [<code>solve_r_for_factors(r, N, ..)</code>](../docs/factoring/general/postprocessing/ekera/solve_r_for_factors.md) to solve $r$ for the prime factors of $N$ using the post-processing in [[E21p]](https://doi.org/10.1007/s11128-021-03069-1):

In [44]:
from quaspy.orderfinding.general.postprocessing.ekera import solve_j_for_r;
from quaspy.factoring.general.postprocessing.ekera import solve_r_for_factors;

rp = solve_j_for_r(j, m, l, g, accept_multiple=True);

print("Solving j for a multiple r' of the order r yielded:", result);

if (rp != 1) and (r % rp == 0):
  print("[ OK ] A multiple r' of the order r was successfully recovered.\n");

  result = solve_r_for_factors(r, N);

  print("Solving r for the complete factorization of N yielded:", result);

  if result == primes:
    print("[ OK ] All prime factors of N were successfully recovered.");
  else:
    print("[FAIL] Failed to completely factor N.");

else:
  print("[FAIL] Failed to recover a multiple r' of the order r.");

Solving j for a multiple r' of the order r yielded: {95530818810920301323859954721354145641670346076988832890618641281060557058014973554951390168583823688471547392871463828302760319229799523856036766929176887827734618137680833804661535495451153382843942006931810404176944464241129459712370074271324390660380728174818825782833957033171113651225663076780920467849, 97857543407927108223976823666216924412885200224964459135278027351739401694392321676669136985358183756653565248925947644724612284283337105672741607915407458372068883562261016667136123365453015285069502858535738893604886644778944039363396403852291342307810410482009805706361658322521650907301518396712507141789, 159597504227174565597698629863466563574714132862427668822594835820203283398809094526979263623109822457236741195185169544775182177049623665555153891296800136064564745412141368327254360766883968539202457353783344315528837573545344644329438989225872602933813276961807069734861025132810023540242969225916636453013, 1683506312627303

## 3. Simulating order finding in $\mathbb Z_N^*$ heuristically to sample $g$ and $r$

In addition to the [<code>sample_r_given_N(N, factors)</code>](../docs/factoring/sampling/sample_r_given_N.md) function used above, [Quaspy](https://github.com/ekera/quaspy) also provides a function [<code>sample_g_r_given_N(N, N_factors, ..)</code>](../docs/factoring/sampling/sample_g_r_given_N.md) for sampling an element $g$ uniformly at random from $\mathbb Z_N^*$ and heuristically computing and returning its order $r$ alongside $g$.

This function requires that the factorization of $N$ is known, as is the case here given that we selected the factors of $N$. (If the function is also fed the factorization of $p_i - 1$ for $i \in [1, n] \cap \mathbb Z$ then the function can be made exact, as explained in the [documentation](../docs/factoring/sampling/sample_r_given_N.md).)

Below, we use this function to simulate the initial classical pre-processing step of Shor's algorithm where an element $g$ is selected uniformly at random from $\mathbb Z_N^*$. This yields $g$ and $r$:

In [45]:
from quaspy.factoring.sampling import sample_g_r_given_N;

[g, r] = sample_g_r_given_N(N, factors);

print("Sampled g =", g, "\n");
print("Sampled r =", r, "\n");

Sampled g = 3876987545641260836785758971711535785798615470845956328028859573837235065571783414585480401424303723862348476439493100366545478811356136221794870776356624997405990886636965315042919975477918045120181981126655257994295971829747885614950815057855322578993086401865916936159777386303980394701244951633432098538247663273878290545506577775743813346281379763066651101851368606996198369648056681381587683765946833353161055529856724702678828129051857569672915775553414932349252782915482732639130042590983709765265559682656617370769815593275778245102478131649055325241306046435296005978383557451238233306014249652508004809335177135795042042219936270849282334827874941948781867534583178079969796024908566042120462890565907978478733648367926149403440092736481308149537660422359381075583830314376093042506929317393867507383007133327178536405872216313141660188045208066314987170666598275779117250891887974744369857527801683532143442052065530976014790950354312000094463273809275980903678587000341123269

### 3.1. Simulating the quantum part of Shor's order-finding algorithm to sample $j$
[Quaspy](https://github.com/ekera/quaspy) provides a function [<code>sample_j_given_r(r, m, l, ..)</code>](../docs/orderfinding/general/sampling/sample_j_given_r.md) for simulating Shor's order-finding algorithm exactly (up to arbitrary precision) for a given order $r$. Below, we use the this function to simulate running the algorithm for $r$ with a control register of $m + \ell$ bits.

This yields a frequency $j$ sampled from the probability distribution induced by the algorithm:

In [46]:
from quaspy.orderfinding.general.sampling import sample_j_given_r;

m = N.bit_length() - 1;
l = m; # Ensures that r^2 < 2^(m + l) as r < 2^m;

j = sample_j_given_r(r, m, l);

print("Sampled j =", j);

Sampled j = 2544612963382043579870044826098796079812022879054417117376058651273964516897629393946505514306892683243622619180697637504829501691630001445218939459600336550333232021210062785773857469009415306378417068500985046829387155407585314614823885774627303656201372224839354247306749406224441563617558689920414529446868846383460848539223583299826505891576436225507974744484396738428498512101310532751071708554583919831364255621199829300324044964098560953479143773251848538614264605235892149926292631875965084809123328706488036162850747452913505376896471493714586547196342417116509432072209069863733210357754989423828410700357023668820426729212736008384201775929092960448356708191280844700982441927102102290647241893576544604082835493182990777446820418220582243293690811232402130677507709027385381695664183892456262154128651116824454487405471390494542914581363576652394340322089334841699093833694704554078438942939310134559445046998602796404576363332207943166221686767549370339687305480914962381531

### 3.2. Solving $j$ for the complete factorization of $N$
We may then e.g. use the [<code>solve_j_for_factors_mod_N(j, m, l, g, N, ..)</code>](../docs/factoring/general/postprocessing/ekera/solve_j_for_factors_mod_N.md) convenience function provided by [Quaspy](https://github.com/ekera/quaspy) to first solve $j$ for a positive integer multiple of $r$ using the lattice-based post-processing from [[E22p]](https://doi.org/10.48550/arXiv.2201.07791), and to then solve $r$ for the prime factors of $N$ using the post-processing in [[E21p]](https://doi.org/10.1007/s11128-021-03069-1):

In [47]:
from quaspy.factoring.general.postprocessing.ekera import solve_j_for_factors_mod_N;

result = solve_j_for_factors_mod_N(j, m, l, g, N);

print("Solving for all prime factors of N via [E21b] yielded:", result);

if result == primes:
  print("[ OK ] All prime factors of N were successfully recovered.");
else:
  print("[FAIL] Failed to completely factor the integer N.");

Solving for all prime factors of N via [E21b] yielded: {97857543407927108223976823666216924412885200224964459135278027351739401694392321676669136985358183756653565248925947644724612284283337105672741607915407458372068883562261016667136123365453015285069502858535738893604886644778944039363396403852291342307810410482009805706361658322521650907301518396712507141789, 159597504227174565597698629863466563574714132862427668822594835820203283398809094526979263623109822457236741195185169544775182177049623665555153891296800136064564745412141368327254360766883968539202457353783344315528837573545344644329438989225872602933813276961807069734861025132810023540242969225916636453013, 168350631262730332687538714284560719119279569826034938392924032400500455940600742310990674827135306020817881394412391528052752331741502604980453974491861744841711699370925343410654146322361748119481938347388263785805051801699247628991106241477009557843089612241238690494426289462077577617907621979753217304197, 103055157400

### 3.3. Solving $j$ for $r$
We may also use the use the [<code>solve_j_for_r_mod_N(j, m, l, g, N, ..)</code>](../docs/factoring/general/postprocessing/ekera/solve_j_for_r_mod_N.md) convenience function provided by [Quaspy](https://github.com/ekera/quaspy) to solve $j$ for $r$ using the lattice-based post-processing from [[E22p]](https://doi.org/10.48550/arXiv.2201.07791).

In [48]:
from quaspy.orderfinding.general.postprocessing.ekera import solve_j_for_r_mod_N;

result = solve_j_for_r_mod_N(j, m, l, g, N);

print("Solving j for r via [E22b] yielded:", result);

if result == r:
  print("[ OK ] The order r was successfully recovered.");
else:
  print("[FAIL] Failed to recover the order r.");

Solving j for r via [E22b] yielded: 1193722578173983528514828101771913009037464219906616541928652251843518572778776139365835290895804097853849635810679438826825965463446997135572699864380914673014971642519344751994575862317433850895695600687784946362133770458783574466008538470489726202009045412894388983897949974797934367509224224170914337616011502871996580259899730792803228757954933066395060673371986777698880259445976653151584869425566025471786028092585061156236633038618840955138170020271936196663712782120647757010536541135314224360667598098713781168003591140256926214268702855670524821159270576589257021359304241677321476581880134198544012174486520649118689086363664894617399087853728990558289136992625029181096045235388615874824693762834654731885788240248362498038376939104815168052253974950089527025906971611244928373756143894282273114936577823220587416156408584821623272474965468136413823081748738612758004877931709675033307358476381997977489991775118837990933781762734785347674272465268465

### 3.4. Splitting $N$ given $g$ and $r$ using Shor's original algorithm

We may then split $N$ given $g$ and $r$ using Shor's original algorithm in [[Shor94]](https://doi.org/10.1109/SFCS.1994.365700).

To this end, we use the [<code>split_N_given_g_r(g, r, N)</code>](../docs/factoring/general/postprocessing/shor/split_N_given_g_r.md) convenience function provided by [Quaspy](https://github.com/ekera/quaspy).

In [49]:
from quaspy.factoring.general.postprocessing.shor import split_N_given_g_r;

result = split_N_given_g_r(g, r, N);

print("Attempting to split N using Shor's algorithm yielded:", result);

Attempting to split N using Shor's algorithm yielded: {217355980029974734293712082894904221369763804963745644501320627817354815635268118461745663717935581471289088520565916898417980208037351812107795235199559037568809716945561752797789497194616814218607239196001638866157824616244833240619637632192014635218713837806297576862904977592523696800847513397822490634961519684111492803258067457320311364997780481816880557801029863730255680341108793308095102802005512302813615200823444191269621512301271365042982813540598303436076985835220788699919309141878034252852975264401023272994452112437169305758761623550833767466798606969297218305171874148921609144249073440415016928640174282526769220608865497122639625600630467419811930194881898572930122077496123401056376236793421358266297927826423479052027126849026076148124505704677997611518938662395386902350490344974856259794009283609276637018221568800612706000784458264393807219444474849012027584264250904593160863914782243009925289722519715451641076684067442